In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Gerando dados sintéticos
np.random.seed(42)
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Adicionando bias
X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# Implementação da Elastic Net manualmente
def elastic_net(X, y, alpha=0.1, l1_ratio=0.5, lr=0.01, epochs=1000):
    m, n = X.shape
    theta = np.random.randn(n, 1)
    for epoch in range(epochs):
        gradients = (2/m) * X.T.dot(X.dot(theta) - y) + alpha * (
            l1_ratio * np.sign(theta) + (1 - l1_ratio) * theta)
        theta -= lr * gradients
    return theta

# Treinando modelo
elastic_theta = elastic_net(X_train_b, y_train)

# Predições
y_train_pred = X_train_b.dot(elastic_theta)
y_test_pred = X_test_b.dot(elastic_theta)

# Erro
train_error = np.mean((y_train_pred - y_train) ** 2)
test_error = np.mean((y_test_pred - y_test) ** 2)

print("Erro de Treino Elastic Net:", train_error)
print("Erro de Teste Elastic Net:", test_error)

# Plotando resultados
plt.scatter(X_train, y_train, label="Treino")
plt.scatter(X_test, y_test, label="Teste", color='red')
plt.plot(X_train, y_train_pred, color='blue', linewidth=2, label="Elastic Net")
plt.legend()
plt.xlabel("X")
plt.ylabel("y")
plt.title("Elastic Net Manual")
plt.show()

# Implementação manual da Decision Tree
class DecisionTree:
    def __init__(self, depth=3):
        self.depth = depth
        self.tree = None
    
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)
    
    def _build_tree(self, X, y, depth):
        if depth >= self.depth or len(np.unique(y)) == 1:
            return np.mean(y)
        
        best_feature, best_threshold = self._find_best_split(X, y)
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask
        
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        
        return (best_feature, best_threshold, left_subtree, right_subtree)
    
    def _find_best_split(self, X, y):
        best_feature, best_threshold, best_loss = None, None, float('inf')
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                
                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue
                
                left_loss = np.var(y[left_mask]) * np.sum(left_mask)
                right_loss = np.var(y[right_mask]) * np.sum(right_mask)
                loss = left_loss + right_loss
                
                if loss < best_loss:
                    best_loss = loss
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold
    
    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])
    
    def _predict_one(self, x, node):
        if not isinstance(node, tuple):
            return node
        feature, threshold, left, right = node
        return self._predict_one(x, left) if x[feature] <= threshold else self._predict_one(x, right)

# Treinando a árvore
X_train_tree = X_train.reshape(-1, 1)
y_train_tree = y_train.ravel()
dt = DecisionTree(depth=3)
dt.fit(X_train_tree, y_train_tree)

# Predições
y_train_pred_tree = dt.predict(X_train_tree)
y_test_pred_tree = dt.predict(X_test)

# Erro
train_error_tree = np.mean((y_train_pred_tree - y_train_tree) ** 2)
test_error_tree = np.mean((y_test_pred_tree - y_test.ravel()) ** 2)

print("Erro de Treino Decision Tree:", train_error_tree)
print("Erro de Teste Decision Tree:", test_error_tree)

# Plotando resultados
plt.scatter(X_train, y_train, label="Treino")
plt.scatter(X_test, y_test, label="Teste", color='red')
plt.scatter(X_train, y_train_pred_tree, color='blue', label="Decision Tree")
plt.legend()
plt.xlabel("X")
plt.ylabel("y")
plt.title("Decision Tree Manual")
plt.show()

# Implementação manual do SVM (SVC - Kernel Linear)
def hinge_loss(X, y, w, b, C):
    margins = 1 - y * (X.dot(w) + b)
    loss = np.maximum(0, margins)
    return 0.5 * np.sum(w ** 2) + C * np.sum(loss)

def sgd_svm(X, y, C=1.0, lr=0.01, epochs=1000):
    m, n = X.shape
    w = np.zeros(n)
    b = 0
    
    for epoch in range(epochs):
        for i in range(m):
            if y[i] * (X[i].dot(w) + b) < 1:
                w -= lr * (w - C * y[i] * X[i])
                b -= lr * (-C * y[i])
            else:
                w -= lr * w
    return w, b

# Convertendo labels para -1 e 1
y_svm = np.where(y_train > np.median(y_train), 1, -1)

# Treinando SVM
w_svm, b_svm = sgd_svm(X_train, y_svm)

# Predições
y_train_pred_svm = np.sign(X_train.dot(w_svm) + b_svm)
y_test_pred_svm = np.sign(X_test.dot(w_svm) + b_svm)

# Erro
train_error_svm = np.mean(y_train_pred_svm != y_svm)
test_error_svm = np.mean(y_test_pred_svm != np.where(y_test > np.median(y_test), 1, -1))

print("Erro de Treino SVM:", train_error_svm)
print("Erro de Teste SVM:", test_error_svm)
